# Avellaneda Parametrisation

We repeat the Radon-Nikodyn Derivative calculation using the parametric optimisation based Avellaneda.

In [ ]:
using Pkg
Pkg.activate(".")

using DiffFusion
using LinearAlgebra
using Interpolations
using OrderedCollections
using Printf
using NonlinearSolve
using Roots
using StatsBase
using StatsPlots
using YAML

default(fmt = :png)

In [ ]:
include("plots.jl")
include("avellaneda.jl")

In [ ]:
dict_list = YAML.load_file("model.yaml"; dicttype=OrderedDict{String,Any})
obj_dict = DiffFusion.deserialise_from_list(dict_list)

times = 0.0:1.0:10.0
n_paths = 2^13

sim = DiffFusion.simple_simulation(obj_dict["md/G3"], obj_dict["ch/STD"], times, n_paths, with_progress_bar=false)
path = DiffFusion.path(sim, DiffFusion.Examples.term_structures(obj_dict), obj_dict["ct/STD"]);

In [ ]:
e = ones(length(path)); # set RW measure equal to RN measure for initial plots

In [ ]:
usd_2y = DiffFusion.LiborRate(2.0, 2.0, 4.0, "USD:SOFR")(path)
usd_5y = DiffFusion.LiborRate(5.0, 5.0, 10.0, "USD:SOFR")(path)
usd_10y = DiffFusion.LiborRate(10.0, 10.0, 20.0, "USD:SOFR")(path)

p1 = plot_density(usd_2y,  xlabel = "USD_2y-2y")
p2 = plot_density(usd_5y,  xlabel = "USD_5y-5y")
p3 = plot_density(usd_10y, xlabel = "USD_10y-10y")

p = plot(p1, p2, p3, layout=(1,3), size = (900, 300),)

In [ ]:
eur_2y = DiffFusion.LiborRate(2.0, 2.0, 4.0, "EUR:ESTR")(path)
eur_5y = DiffFusion.LiborRate(5.0, 5.0, 10.0, "EUR:ESTR")(path)
eur_10y = DiffFusion.LiborRate(10.0, 10.0, 20.0, "EUR:ESTR")(path)

p1 = plot_density(eur_2y,  xlabel = "EUR_2y-2y")
p2 = plot_density(eur_5y,  xlabel = "EUR_5y-5y")
p3 = plot_density(eur_10y, xlabel = "EUR_10y-10y")

p = plot(p1, p2, p3, layout=(1,3), size = (900, 300),)

In [ ]:
gbp_2y = DiffFusion.LiborRate(2.0, 2.0, 4.0, "GBP:SONIA")(path)
gbp_5y = DiffFusion.LiborRate(5.0, 5.0, 10.0, "GBP:SONIA")(path)
gbp_10y = DiffFusion.LiborRate(10.0, 10.0, 20.0, "GBP:SONIA")(path)

p1 = plot_density(gbp_2y,  xlabel = "GBP_2y-2y")
p2 = plot_density(gbp_5y,  xlabel = "GBP_5y-5y")
p3 = plot_density(gbp_10y, xlabel = "GBP_10y-10y")

p = plot(p1, p2, p3, layout=(1,3), size = (900, 300),)

In [ ]:
eur_usd_2y = DiffFusion.Asset(2.0, "EUR-USD")(path)
eur_usd_5y = DiffFusion.Asset(5.0, "EUR-USD")(path)
eur_usd_10y = DiffFusion.Asset(10.0, "EUR-USD")(path)

p1 = plot_density(eur_usd_2y,  xlabel = "EUR-USD 2y")
p2 = plot_density(eur_usd_5y,  xlabel = "EUR-USD 5y")
p3 = plot_density(eur_usd_10y, xlabel = "EUR-USD 10y")

p = plot(p1, p2, p3, layout=(1,3), size = (900, 300),)

In [ ]:
gbp_usd_2y = DiffFusion.Asset(2.0, "GBP-USD")(path)
gbp_usd_5y = DiffFusion.Asset(5.0, "GBP-USD")(path)
gbp_usd_10y = DiffFusion.Asset(10.0, "GBP-USD")(path)

p1 = plot_density(gbp_usd_2y,  xlabel = "GBP-USD 2y")
p2 = plot_density(gbp_usd_5y,  xlabel = "GBP-USD 5y")
p3 = plot_density(gbp_usd_10y, xlabel = "GBP-USD 10y")

p = plot(p1, p2, p3, layout=(1,3), size = (900, 300),)

### Calibrate to Single Riskfactor

In [ ]:
res = avellaneda(reshape(usd_10y, (:,1)), [0.03, ], [0.05, ])
println(res.convergence)
plot_density(res.w, xlabel = "Radon-Nikody Derivative", plot_size = (300, 300),)

In [ ]:
plot_densities(usd_10y, res.w, xlabel = "USD_10y-10y")

### Calibrate to Term Structure of Risk Factors

In [ ]:
res = avellaneda(hcat(eur_2y, eur_5y, eur_10y), [0.03, 0.025, 0.02, ], [0.015, 0.025, 0.03])
println(res.convergence)
plot_density(res.w, xlabel = "Radon-Nikody Derivative", plot_size = (300, 300),)

In [ ]:
p1 = plot_densities(eur_2y, res.w,  xlabel = "EUR_2y-2y")
p2 = plot_densities(eur_5y, res.w,  xlabel = "EUR_5y-5y")
p3 = plot_densities(eur_10y, res.w, xlabel = "EUR_10y-10y")
p = plot(p1, p2, p3, layout=(3,1), size = (700, 900),)

### Calibrate Slice of Hybrid Model

In [ ]:
res = avellaneda(
    hcat(usd_10y, eur_10y, gbp_10y, eur_usd_10y, gbp_usd_10y,),
    [    0.040,   0.020,   0.030,   1.10,        1.15,],
    [    0.040,   0.040,   0.040,   0.30,        0.30,],
)
println(res.convergence)
plot_density(res.w, xlabel = "Radon-Nikody Derivative", plot_size = (300, 300),)

In [ ]:
p1 = plot_densities(usd_10y, res.w, xlabel = "USD_10y-10y")
p2 = plot_densities(eur_10y, res.w, xlabel = "EUR_10y-10y")
p3 = plot_densities(gbp_10y, res.w, xlabel = "GBP_10y-10y")
p4 = plot_densities(eur_usd_10y, res.w, xlabel = "EUR-USD_10y-10y")
p5 = plot_densities(gbp_usd_10y, res.w, xlabel = "GBP-USD_10y-10y")
p = plot(p1, p2, p3, p4, p5, layout=(5,1), size = (700, 1500),)

### Full Hybrid Model Calibration

In [ ]:
res = avellaneda(
    hcat(
        usd_2y,     usd_5y,     usd_10y,
        eur_2y,     eur_5y,     eur_10y,
        gbp_2y,     gbp_5y,     gbp_10y,
        eur_usd_2y, eur_usd_2y, eur_usd_10y,
        gbp_usd_2y, gbp_usd_2y, gbp_usd_10y,
    ),
    # E[X]
    [
        0.040,      0.040,      0.040, 
        0.020,      0.020,      0.020, 
        0.030,      0.030,      0.030, 
        1.100,      1.100,      1.100, 
        1.150,      1.150,      1.150, 
    ],
    # Std
    [
        0.020,      0.025,      0.040,
        0.020,      0.025,      0.040,
        0.020,      0.025,      0.040,
        0.150,      0.250,      0.300,
        0.150,      0.250,      0.300,
    ],
    # [
    #     0.0,      0.0,      0.0,
    #     0.0,      0.0,      0.0,
    #     0.0,      0.0,      0.0,
    #     0.0,      0.0,      0.0,
    #     0.0,      0.0,      0.0,
    # ],
)
println(res.convergence)
plot_density(res.w, xlabel = "Radon-Nikody Derivative", plot_size = (300, 300),)

In [ ]:
res.details.stats

In [ ]:
fs = 6

p1 = plot_densities(usd_2y,  res.w, font_size = fs, xlabel = "USD_2y-2y")
p2 = plot_densities(usd_5y,  res.w, font_size = fs, xlabel = "USD_5y-5y")
p3 = plot_densities(usd_10y, res.w, font_size = fs, xlabel = "USD_10y-10y")

p_usd = plot(p1, p2, p3,
    layout=(1,3),
    size = (1500, 300),
    left_margin = 5Plots.mm,  # adjust this if xaxis label is cut off
)

p1 = plot_densities(eur_2y,  res.w, font_size = fs, xlabel = "EUR_2y-2y")
p2 = plot_densities(eur_5y,  res.w, font_size = fs, xlabel = "EUR_5y-5y")
p3 = plot_densities(eur_10y, res.w, font_size = fs, xlabel = "EUR_10y-10y")

p_eur = plot(p1, p2, p3,
    layout=(1,3),
    size = (1500, 300),
    left_margin = 5Plots.mm,  # adjust this if xaxis label is cut off
)

p1 = plot_densities(gbp_2y, res.w, font_size = fs,  xlabel = "GBP_2y-2y")
p2 = plot_densities(gbp_5y, res.w, font_size = fs,  xlabel = "GBP_5y-5y")
p3 = plot_densities(gbp_10y, res.w, font_size = fs, xlabel = "GBP_10y-10y")

p_gbp = plot(p1, p2, p3,
    layout=(1,3),
    size = (1500, 300),
    left_margin = 5Plots.mm,  # adjust this if xaxis label is cut off
)

p1 = plot_densities(eur_usd_2y,  res.w, font_size = fs, xlabel = "EUR-USD_2y")
p2 = plot_densities(eur_usd_5y,  res.w, font_size = fs, xlabel = "EUR-USD_5y")
p3 = plot_densities(eur_usd_10y, res.w, font_size = fs, xlabel = "EUR-USD_10y")

p_eur_usd = plot(p1, p2, p3,
    layout=(1,3),
    size = (1500, 300),
    left_margin = 5Plots.mm,  # adjust this if xaxis label is cut off
)

p1 = plot_densities(gbp_usd_2y, res.w, font_size = fs,  xlabel = "GBP-USD_2y")
p2 = plot_densities(gbp_usd_5y, res.w, font_size = fs,  xlabel = "GBP-USD_5y")
p3 = plot_densities(gbp_usd_10y, res.w, font_size = fs, xlabel = "GBP-USD_10y")

p_gbp_usd = plot(p1, p2, p3,
    layout=(1,3),
    size = (1500, 300),
    left_margin = 5Plots.mm,  # adjust this if xaxis label is cut off
)

# full plot
p = plot(p_usd, p_eur, p_gbp, p_eur_usd, p_gbp_usd,
    layout=(5,1),
    size = (1500, 1500),
    left_margin = 5Plots.mm,  # adjust this if xaxis label is cut off
)

### Test Densty *Interpolation* and *Extrapolation*

In [ ]:
res = avellaneda(hcat(usd_2y, usd_10y), [0.03, 0.02, ], [0.015, 0.03])
println(res.convergence)
plot_density(res.w, xlabel = "Radon-Nikody Derivative", plot_size = (300, 300),)

In [ ]:
fs = 6

p1 = plot_densities(usd_2y,  res.w, font_size = fs, xlabel = "USD_2y-2y")
p2 = plot_densities(usd_5y,  res.w, font_size = fs, xlabel = "USD_5y-5y")
p3 = plot_densities(usd_10y, res.w, font_size = fs, xlabel = "USD_10y-10y")

p_usd = plot(p1, p2, p3,
    layout=(1,3),
    size = (1500, 300),
    left_margin = 5Plots.mm,  # adjust this if xaxis label is cut off
)